# Zadanie: kanały sprzedaży

In [7]:
import pandas as pd
import datetime as dt
import os
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px

class DataLoader:
    def __init__(self):
        self.db_folder = 'db'  # Folder główny z danymi
        self.transactions_dir = os.path.join(self.db_folder, 'transactions')  # Podfolder z transakcjami
        self.transactions = self.load_transactions()
        self.cc = pd.read_csv(os.path.join(self.db_folder, 'country_codes.csv'), index_col=0)
        self.customers = pd.read_csv(os.path.join(self.db_folder, 'customers.csv'), index_col=0)
        self.prod_info = pd.read_csv(os.path.join(self.db_folder, 'prod_cat_info.csv'))
        self.merged_data = None
        
    def load_transactions(self):
        if not os.path.exists(self.transactions_dir):
            raise FileNotFoundError(f"Folder '{self.transactions_dir}' nie istnieje!")
        
        transactions = pd.DataFrame()
        
        # Wczytaj wszystkie pliki transakcji
        for filename in os.listdir(self.transactions_dir):
            if filename.startswith('transactions-') and filename.endswith('.csv'):
                file_path = os.path.join(self.transactions_dir, filename)
                try:
                    df = pd.read_csv(file_path, index_col=0)
                    transactions = pd.concat([transactions, df])
                except Exception as e:
                    print(f"Błąd przy wczytywaniu {filename}: {str(e)}")
        
        if transactions.empty:
            raise ValueError("Nie wczytano żadnych danych transakcyjnych!")
        
        # Konwersja daty
        def convert_date(x):
            try:
                return dt.datetime.strptime(x, '%d-%m-%Y')
            except:
                try:
                    return dt.datetime.strptime(x, '%d/%m/%Y')
                except:
                    return pd.NaT
        
        transactions['tran_date'] = transactions['tran_date'].apply(convert_date)
        transactions = transactions.dropna(subset=['tran_date'])  # Usuń wiersze z błędnymi datami
        
        return transactions
    
    def merge_data(self):
        # Połączenie danych o produktach
        df = self.transactions.merge(
            self.prod_info.drop_duplicates(subset=['prod_cat_code'])[['prod_cat_code', 'prod_cat']],
            on='prod_cat_code',
            how='left'
        )
        
        df = df.merge(
            self.prod_info.drop_duplicates(subset=['prod_sub_cat_code'])[['prod_sub_cat_code', 'prod_subcat']],
            left_on='prod_subcat_code',
            right_on='prod_sub_cat_code',
            how='left'
        )
        
        # Połączenie danych o klientach z krajami
        customers_with_country = self.customers.merge(
            self.cc,
            left_on='country_code',
            right_index=True,
            how='left'
        )
        
        # Finalne połączenie
        df = df.merge(
            customers_with_country,
            left_on='cust_id',
            right_on='customer_Id',
            how='left'
        )
        
        # Dodatkowe przetwarzanie
        df['day_of_week'] = df['tran_date'].dt.day_name()
        df['is_positive'] = df['total_amt'] > 0
        
        self.merged_data = df

# Inicjalizacja danych
try:
    data_loader = DataLoader()
    data_loader.merge_data()
    df = data_loader.merged_data
    
    # Sprawdzenie czy dane zostały wczytane
    if df.empty:
        raise ValueError("Brak danych po połączeniu!")
    print(f"Pomyślnie wczytano {len(df)} rekordów")
    
except Exception as e:
    print(f"Błąd inicjalizacji danych: {str(e)}")
    raise

# Inicjalizacja aplikacji Dash
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# Funkcje do renderowania zakładek
def render_tab1():
    return html.Div([
        html.H1('Sprzedaż globalna', style={'text-align': 'center'}),
        html.Div([
            dcc.DatePickerRange(
                id='sales-range',
                start_date=df['tran_date'].min(),
                end_date=df['tran_date'].max(),
                display_format='YYYY-MM-DD'
            )
        ], style={'width': '100%', 'text-align': 'center'}),
        html.Div([
            html.Div([dcc.Graph(id='bar-sales')], style={'width': '50%'}),
            html.Div([dcc.Graph(id='choropleth-sales')], style={'width': '50%'})
        ], style={'display': 'flex'})
    ])

def render_tab2():
    grouped = df[df['is_positive']].groupby('prod_cat')['total_amt'].sum()
    fig_pie = go.Figure(
        data=[go.Pie(labels=grouped.index, values=grouped.values)],
        layout=go.Layout(title='Udział grup produktów w sprzedaży')
    )
    
    return html.Div([
        html.H1('Produkty', style={'text-align': 'center'}),
        html.Div([
            html.Div([
                dcc.Graph(id='pie-prod-cat', figure=fig_pie)
            ], style={'width': '50%'}),
            html.Div([
                dcc.Dropdown(
                    id='prod_dropdown',
                    options=[{'label': cat, 'value': cat} for cat in df['prod_cat'].unique()],
                    value=df['prod_cat'].unique()[0]
                ),
                dcc.Graph(id='barh-prod-subcat')
            ], style={'width': '50%'})
        ], style={'display': 'flex'})
    ])

def render_tab3():
    return html.Div([
        html.H1('Kanały sprzedaży', style={'text-align': 'center'}),
        html.Div([
            dcc.DatePickerRange(
                id='store-range',
                start_date=df['tran_date'].min(),
                end_date=df['tran_date'].max(),
                display_format='YYYY-MM-DD'
            )
        ], style={'width': '100%', 'text-align': 'center'}),
        html.Div([
            dcc.Graph(id='sales-by-day-store'),
            dcc.Graph(id='gender-distribution'),
            dcc.Graph(id='avg-sale-value'),
            dcc.Graph(id='geo-distribution')
        ], style={'display': 'flex', 'flex-wrap': 'wrap'})
    ])

# Layout aplikacji
app.layout = html.Div([
    html.Div([
        dcc.Tabs(id='tabs', value='tab-1', children=[
            dcc.Tab(label='Sprzedaż globalna', value='tab-1'),
            dcc.Tab(label='Produkty', value='tab-2'),
            dcc.Tab(label='Kanały sprzedaży', value='tab-3')
        ]),
        html.Div(id='tabs-content')
    ], style={'width': '90%', 'margin': 'auto'})
])

# Callback do przełączania zakładek
@app.callback(
    Output('tabs-content', 'children'),
    [Input('tabs', 'value')]
)
def render_content(tab):
    if tab == 'tab-1':
        return render_tab1()
    elif tab == 'tab-2':
        return render_tab2()
    elif tab == 'tab-3':
        return render_tab3()

# Callback dla zakładki 1
@app.callback(
    [Output('bar-sales', 'figure'),
     Output('choropleth-sales', 'figure')],
    [Input('sales-range', 'start_date'),
     Input('sales-range', 'end_date')]
)
def update_tab1(start_date, end_date):
    filtered = df[(df['tran_date'] >= start_date) & 
                 (df['tran_date'] <= end_date) & 
                 (df['is_positive'])]
    
    # Wykres słupkowy
    sales_by_month = filtered.groupby([pd.Grouper(key='tran_date', freq='M'), 'Store_type'])['total_amt'].sum().unstack()
    bar_fig = go.Figure()
    for store in sales_by_month.columns:
        bar_fig.add_trace(go.Bar(
            x=sales_by_month.index,
            y=sales_by_month[store],
            name=store
        ))
    bar_fig.update_layout(barmode='stack', title='Sprzedaż miesięczna wg kanałów')
    
    # Mapa
    sales_by_country = filtered.groupby('country')['total_amt'].sum().reset_index()
    map_fig = px.choropleth(
        sales_by_country,
        locations='country',
        locationmode='country names',
        color='total_amt',
        title='Sprzedaż wg krajów'
    )
    
    return bar_fig, map_fig

# Callback dla zakładki 2
@app.callback(
    Output('barh-prod-subcat', 'figure'),
    [Input('prod_dropdown', 'value')]
)
def update_tab2(selected_cat):
    filtered = df[(df['prod_cat'] == selected_cat) & (df['is_positive'])]
    sales_by_subcat = filtered.groupby(['prod_subcat', 'Gender'])['total_amt'].sum().unstack().fillna(0)
    
    fig = go.Figure()
    for gender in ['M', 'F']:
        fig.add_trace(go.Bar(
            y=sales_by_subcat.index,
            x=sales_by_subcat[gender],
            name=gender,
            orientation='h'
        ))
    fig.update_layout(barmode='stack', title=f'Sprzedaż wg płci dla: {selected_cat}')
    return fig

# Callback dla zakładki 3
@app.callback(
    [Output('sales-by-day-store', 'figure'),
     Output('gender-distribution', 'figure'),
     Output('avg-sale-value', 'figure'),
     Output('geo-distribution', 'figure')],
    [Input('store-range', 'start_date'),
     Input('store-range', 'end_date')]
)
def update_tab3(start_date, end_date):
    filtered = df[(df['tran_date'] >= start_date) & 
                 (df['tran_date'] <= end_date) & 
                 (df['is_positive'])]
    
    # Wykres 1: Sprzedaż wg dni tygodnia
    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    sales_by_day = filtered.groupby(['day_of_week', 'Store_type'])['total_amt'].sum().unstack().reindex(days_order)
    fig1 = px.bar(sales_by_day, barmode='group', title='Sprzedaż wg dni tygodnia')
    
    # Wykres 2: Rozkład płci
    gender_dist = filtered.groupby(['Store_type', 'Gender']).size().unstack().fillna(0)
    fig2 = px.bar(gender_dist, barmode='group', title='Rozkład płci klientów')
    
    # Wykres 3: Średnia wartość transakcji
    avg_sale = filtered.groupby('Store_type')['total_amt'].mean().reset_index()
    fig3 = px.bar(avg_sale, x='Store_type', y='total_amt', title='Średnia wartość transakcji')
    
    # Wykres 4: Rozkład geograficzny
    geo_sales = filtered.groupby(['Store_type', 'country'])['total_amt'].sum().unstack().fillna(0)
    fig4 = px.choropleth(
        geo_sales.T.reset_index(),
        locations='country',
        locationmode='country names',
        color=geo_sales.index[0],
        title='Sprzedaż wg krajów'
    )
    
    return fig1, fig2, fig3, fig4

if __name__ == '__main__':
    app.run_server(debug=True)

Pomyślnie wczytano 23053 rekordów
